In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import psycopg2
import pandas as pd
import seaborn as sns
from pathlib import Path
import matplotlib.pyplot as plt
%matplotlib inline

sns.set(style = 'darkgrid')

In [ ]:
con = psycopg2.connect(dbname='mimic', user='sudarshan', host='/var/run/postgresql')

In [ ]:
query = """
select count(*) as n_notes from data group by chartinterval order by chartinterval;
"""

df = pd.read_sql_query(query, con)
intervals = ['0 < t < 12', '12 < t < 24', '24 < t < 36', '36 < t < 48', '48 < t < 60', '60 < t < 72', '72 < t < 84', '84 < t < 96', '96 < t < 108', '108 < t < 120', '120 < t 132', '132 < t < 144', '144 < t < 156', '156 < t < 168', '168 < t < 180', 't > 180']
df['hours'] = intervals

df

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))

sns.barplot(x='hours', y='n_notes', data=df, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(),rotation=45, ha='right', fontweight='bold', fontsize=12)
ax.set_xlabel("Time between note chart time and ICU admit time (hours)\n 't' is the time-to-ICU admit time", fontweight='bold', fontsize=15)
ax.set_ylabel('Number of notes', fontweight='bold', fontsize=15)
ax.set_title('Distribution of notes w.r.t chart time and ICU admit time', fontweight='bold', fontsize=15)
ax.set_ylim(top=17000)

ax.text(7, 14000, 'Total number of notes = 52,391', fontweight='bold', fontsize=15, ha='center', va='bottom')

for index, row in df.iterrows():
    ax.text(index, row['n_notes'], str(row['n_notes']), color='black', ha='center', va='bottom', fontweight='bold', fontsize=12)

In [ ]:
query = """
select hadm_id, count(*) as n_notes from data group by hadm_id order by hadm_id;
"""
df = pd.read_sql_query(query, con)
df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))

sns.distplot(df['n_notes'], kde=False, ax=ax, rug=True, bins=100)
ax.set_xlabel('Number of notes', fontweight='bold', fontsize=15)
ax.set_ylabel('Number of hospital admissions', fontweight='bold', fontsize=15)
ax.set_title('Histogram of note distribution Vs number of hospital admissions', fontweight='bold', fontsize=15)
ax.set_ylim(0, 5500)
ax.set_xlim(0, 40)

ax.text(20, 4500, 'Total number of hospital admissions = 10,610', fontweight='bold', fontsize=15, ha='center', va='bottom')

In [ ]:
query = """
select hadm_id, avg(wait_period) as avg_wait_period from data group by hadm_id order by hadm_id;
"""
df = pd.read_sql_query(query, con)
df.head()

In [ ]:
df['avg_wait_period'].min(), df['avg_wait_period'].mean(), df['avg_wait_period'].max()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))

sns.distplot(df['avg_wait_period'], kde=False, ax=ax, bins=100)
ax.set_xlabel('Average wait_period per admission (hours)', fontweight='bold', fontsize=15)
ax.set_ylabel('Number of hospital admissions', fontweight='bold', fontsize=15)
ax.set_title('Histogram of average wait_period Vs number of hospital admissions', fontweight='bold', fontsize=15)
ax.set_xlim(0, 600)

ax.text(300, 3000, 'Total number of hospital admissions = 10,610', fontweight='bold', fontsize=15, ha='center', va='bottom')

In [ ]:
query = """
with test as (select subject_id, count(distinct hadm_id) as n_adms from data group by subject_id order by subject_id) select n_adms, count(*) from test group by n_adms order by n_adms;
"""
df = pd.read_sql_query(query, con)
df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))

sns.barplot(x='n_adms', y='count', data=df, ax=ax)
ax.set_xlabel("Count of number of admissions", fontweight='bold', fontsize=15)
ax.set_ylabel('Number of admissions', fontweight='bold', fontsize=15)
ax.set_title('Distribution of count of number of admissions', fontweight='bold', fontsize=15)
for index, row in df.iterrows():
    ax.text(index, row['count'], str(row['count']), color='black', ha='center', va='bottom', fontweight='bold', fontsize=12)

In [ ]:
con.close()